In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from shapely.wkt import loads
import matplotlib.pyplot as plt

### fix random seed
#random.seed(0)
np.random.seed(0)

In [20]:
case = 'bay_area'

### read OD
edges = pd.read_csv('{}_edges.csv'.format(case))
nodes = pd.read_csv('{}_nodes.csv'.format(case))

### bay area or tokyo
edges['nid_s'] = edges['start_nid']
edges['nid_e'] = edges['end_nid']
nodes['nid'] = nodes['node_id']

print(edges.shape, nodes.shape)
display(edges.head(1))
display(nodes.head(1))

(549008, 18) (224223, 5)


,uniqueid,osmid,length,type,bridge,tunnel,lanes,maxspeed,capacity,fft,weight,start_nid,end_nid,start_sp,end_sp,geometry,nid_s,nid_e
0,0,7714200,175.637,residential,NaN,NaN,1,25.0,950.0,15.715577,18.858693,0,2,1,3,"LINESTRING (-122.7694487 38.4353358, -122.7692...",0,2


,node_id,osmid,lon,lat,nid
0,0,56098817,-122.769449,38.435336,0


In [21]:
removed_edges = edges[edges['length']<=20].copy()
print(edges.shape, removed_edges.shape)
removed_node_grp = {}
grp_id = 0
for edge in removed_edges.itertuples():
    nid_s = getattr(edge, 'nid_s')
    nid_e = getattr(edge, 'nid_e')
    try:
        nid_s_grp = removed_node_grp[nid_s]
    except KeyError:
        nid_s_grp = grp_id
    try:
        nid_e_grp = removed_node_grp[nid_e]
    except KeyError:
        nid_e_grp = grp_id
    nid_se_grp_id = min(nid_s_grp, nid_e_grp)
    if (nid_s==755916) and (nid_e==938408):
        print(nid_s, nid_e, nid_s_grp, nid_e_grp)
    removed_node_grp[nid_s] = nid_se_grp_id
    removed_node_grp[nid_e] = nid_se_grp_id
    if nid_se_grp_id == grp_id: grp_id += 1
removed_node_grp_df = pd.DataFrame(removed_node_grp.items(), columns=['nid', 'node_grp'])
removed_node_grp_df['node_grp'] = removed_node_grp_df['node_grp'].apply(lambda x: 'g{}'.format(x))
print(nodes.shape, removed_node_grp_df.shape)
removed_node_grp_df.tail()

(549008, 18) (31444, 18)
(224223, 5) (31720, 2)


,nid,node_grp
31715,224076,g14136
31716,224122,g14137
31717,36587,g14137
31718,224124,g6211
31719,224147,g6211


In [24]:
new_nodes = pd.merge(nodes, removed_node_grp_df, how='left', on='nid')
new_nodes['node_grp'] = np.where(
    pd.isnull(new_nodes['node_grp']), new_nodes['nid'], new_nodes['node_grp'])
new_nodes = new_nodes.groupby('node_grp').agg({'lon': np.mean, 'lat': np.mean}).reset_index()
print(nodes.shape, new_nodes.shape)

(224223, 5) (206426, 3)


In [25]:
new_edges = edges.copy() ### remove those with duplicated new_node_id
new_edges = pd.merge(new_edges, removed_node_grp_df, 
                           how='left', left_on='nid_s', right_on='nid')
new_edges = pd.merge(new_edges, removed_node_grp_df, 
                           how='left', left_on='nid_e', right_on='nid', suffixes=['_ns0', '_ne0'])
new_edges['node_grp_ns0'] = np.where(
    pd.isnull(new_edges['node_grp_ns0']), new_edges['nid_s'], new_edges['node_grp_ns0'])
new_edges['node_grp_ne0'] = np.where(
    pd.isnull(new_edges['node_grp_ne0']), new_edges['nid_e'], new_edges['node_grp_ne0'])
new_edges = new_edges[[
    'nid_s', 'nid_e', 'node_grp_ns0', 'node_grp_ne0', 'length', 'lanes', 'type', 'capacity', 'maxspeed', 'geometry'
]]
new_edges = new_edges.loc[new_edges['node_grp_ns0']!=new_edges['node_grp_ne0']]

### update nodes
### first remove those without links
new_nodes = new_nodes.loc[(
    new_nodes['node_grp'].isin(new_edges['node_grp_ns0'])) | 
    (new_nodes['node_grp'].isin(new_edges['node_grp_ne0']))
]
new_nodes['node_id'] = np.arange(new_nodes.shape[0])

new_edges = pd.merge(new_edges, new_nodes, how='left', left_on='node_grp_ns0', right_on='node_grp')
new_edges = pd.merge(new_edges, new_nodes, how='left', left_on='node_grp_ne0', right_on='node_grp', 
                           suffixes=['_ns', '_ne'])

geometry_list = []
for edge in new_edges.itertuples():
    geometry = getattr(edge, 'geometry').replace('LINESTRING(','').replace(')', '').split(', ')
    geometry = [tuple(xy.split(' ')) for xy in geometry]
    lon_ns, lat_ns = getattr(edge, 'lon_ns'), getattr(edge, 'lat_ns')
    lon_ne, lat_ne = getattr(edge, 'lon_ne'), getattr(edge, 'lat_ne')
    geometry = [(lon_ns, lat_ns)] + geometry[1:-2] + [(lon_ne, lat_ne)]
    geometry_list.append('LINESTRING('+', '.join('{} {}'.format(xy[0], xy[1]) for xy in geometry)+')')
new_edges['geometry'] = geometry_list
new_edges['start_nid'] = new_edges['node_id_ns']
new_edges['end_nid'] = new_edges['node_id_ne']
new_edges['nid_s_old'] = new_edges['nid_s']
new_edges['nid_e_old'] = new_edges['nid_e']
new_edges = new_edges[['start_nid', 'end_nid', 'nid_s_old', 'nid_e_old',
                                  'length', 'lanes', 'type', 'capacity', 'maxspeed', 'geometry']]
new_edges = new_edges.loc[new_edges['start_nid']!=new_edges['end_nid']]

### add attributes
if case=='tokyo':
    new_edges['maxmph'] = new_edges['maxspeed']/1.609
elif case =='bay_area':
    new_edges['maxmph'] = new_edges['maxspeed']
else:
    print('no such case')
new_edges['fft'] = new_edges['length']/(new_edges['maxmph']*1609/3600)
new_edges = new_edges.sort_values(by='fft', ascending=True).drop_duplicates(subset=['start_nid', 'end_nid'], keep='first')

### add link_id
new_edges['link_id'] = np.arange(new_edges.shape[0])
print(new_edges.shape, new_nodes.shape)
display(new_edges.tail(1))
display(new_nodes.tail(1))

(508004, 13) (206426, 4)


,start_nid,end_nid,nid_s_old,nid_e_old,length,lanes,type,capacity,maxspeed,geometry,maxmph,fft,link_id
300108,112667,184406,131779,215043,26716.259,2,secondary,1900.0,25.0,"LINESTRING(-123.1107266 38.6781247, -123.11092...",25.0,2391.013857,508003


,node_grp,lon,lat,node_id
206425,g9999,-122.607582,38.122272,206425


In [26]:
removed_node_grp_df.to_csv('{}_nid_grp_conversion.csv'.format(case), index=False)
new_nodes.to_csv('new_{}_nodes.csv'.format(case), index=False)
new_edges.to_csv('new_{}_links.csv'.format(case), index=False)

In [27]:
### process OD
node_map = nodes[['osmid', 'nid']].merge(
    removed_node_grp_df, how='left', on='nid')
node_map['node_grp'] = np.where(pd.isnull(node_map['node_grp']), node_map['nid'], node_map['node_grp'])
node_map = pd.merge(node_map, new_nodes[['node_grp', 'node_id']], how='left', on='node_grp')
### bay area
if case == 'bay_area':
    node_map_dict = {getattr(n, 'osmid'): getattr(n, 'node_id') for n in node_map.itertuples()}
### tokyo
elif case == 'tokyo':
    node_map_dict = {getattr(n, 'nid'): getattr(n, 'node_id') for n in node_map.itertuples()}
else:
    pass

for od_file in ['../demand_inputs/{}_ods_0'.format(case), 
                '../demand_inputs/{}_ods_1'.format(case), 
                '../demand_inputs/{}_ods_2'.format(case)]:
    print(od_file)
    sub_od = pd.read_csv(od_file+'.csv')
    sub_od['origin_nid'] = sub_od['O'].map(node_map_dict)
    sub_od['destin_nid'] = sub_od['D'].map(node_map_dict)
    ### remove ODs whose nodes have been contracted
    remove_ods = np.isnan(sub_od['origin_nid']) | np.isnan(sub_od['destin_nid'])
    sub_od = sub_od[~remove_ods]
    print('remove {} ods, keep {} ods'.format(np.sum(remove_ods), sub_od.shape[0]))
    sub_od['origin_nid'] = sub_od['origin_nid'].astype(int)
    sub_od['destin_nid'] = sub_od['destin_nid'].astype(int)
    if case == 'bay_area':
        sub_od['hour'] = np.random.choice([6,7,8,9], size=sub_od.shape[0], p=[0.1, 0.4, 0.4, 0.1])
        sub_od['quarter'] = np.random.choice([0, 1, 2, 3], size=sub_od.shape[0])
    elif case == 'tokyo':
        sub_od['hour'] = sub_od['trip_hour']
        sub_od['quarter'] = sub_od['trip_quarter']
    display(sub_od.head(1))
    sub_od[['agent_id', 'O', 'D', 'origin_nid', 'destin_nid', 'hour', 'quarter']].to_csv(
        od_file+'_new.csv', index=False)

../demand_inputs/bay_area_ods_0
remove 0 ods, keep 682111 ods


,agent_id,O,D,origin_nid,destin_nid,hour,quarter
0,0,53019152,240448882,23762,203210,8,1


../demand_inputs/bay_area_ods_1
remove 0 ods, keep 682111 ods


,agent_id,O,D,origin_nid,destin_nid,hour,quarter
0,682111,263061205,65656886,139361,47785,7,3


../demand_inputs/bay_area_ods_2
remove 0 ods, keep 682110 ods


,agent_id,O,D,origin_nid,destin_nid,hour,quarter
0,1364222,53153701,441993862,82264,6839,8,3


In [28]:
### verify OD
if case == 'bay_area': join_nm = 'osmid'
elif case == 'tokyo': join_nm = 'nid'
else: pass
verify_od = sub_od.copy()
verify_od = verify_od.merge(nodes[[join_nm, 'lon', 'lat']], 
                        how='left', left_on='O', right_on=join_nm)
verify_od = verify_od.merge(nodes[[join_nm, 'lon', 'lat']], 
                        how='left', left_on='D', right_on=join_nm, suffixes=['_x', '_y'])
verify_od = verify_od.merge(new_nodes[['node_id', 'lon', 'lat']], 
                        how='left', left_on='origin_nid', right_on='node_id')
verify_od = verify_od.merge(new_nodes[['node_id', 'lon', 'lat']], 
                        how='left', left_on='destin_nid', right_on='node_id', suffixes=['_x_new', '_y_new'])

#display(verify_od.head())
verify_od['lon_x_diff'] = verify_od['lon_x_new'] - verify_od['lon_x']
verify_od['lat_x_diff'] = verify_od['lat_x_new'] - verify_od['lat_x']
verify_od['lon_y_diff'] = verify_od['lon_y_new'] - verify_od['lon_y']
verify_od['lat_y_diff'] = verify_od['lat_y_new'] - verify_od['lat_y']
verify_od[['lon_x_diff', 'lat_x_diff', 'lon_y_diff', 'lat_y_diff']].describe()

,lon_x_diff,lat_x_diff,lon_y_diff,lat_y_diff
count,6.821100e+05,6.821100e+05,6.821100e+05,6.821100e+05
mean,1.380630e-08,-1.537974e-08,1.040615e-07,-7.471832e-08
std,2.362878e-05,1.895902e-05,2.920333e-05,2.276833e-05
min,-2.898800e-04,-2.956429e-04,-2.898800e-04,-2.956429e-04
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,3.629167e-04,2.768571e-04,3.629167e-04,2.768571e-04
